In [8]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Wczytanie danych
df = pd.read_csv(r"C:\Users\tomas\Desktop\Praca dyplomowa\Tweets_Kaggle_Clean.csv")

# Pobranie listy stopwords
nltk.download('stopwords') #biblioteka 
nltk.download('punkt_tab') #biblioteka puntorów 
stop_words = set(stopwords.words('english'))

# Lista nazw linii lotniczych i nieistotnych zwrotów
unnecessary_words = ["united", "american", "delta", "southwest", "jetblue", "virginamerica", "usairways",
                      "flight", "flights", "airline", "plane", "hi", "hello", "australia", "hawaii", "mexico", "atlanta"]

#Aktualizacja 'stopwords'
stop_words.update(unnecessary_words)  # Dodajemy zbędne słowa do listy stopwords

# Funkcja do czyszczenia tekstu
def clean_text(text):
    text = text.lower()  # Małe litery
    text = re.sub(r"http\S+|www\S+", "", text)  # Usunięcie linków
    text = re.sub(r"@\S+|#\S+", "", text)  # Usunięcie mentionów i hashtagów
    tokens = word_tokenize(text)  # Tokenizacja
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return " ".join(tokens)

# Zastosowanie funkcji do danych
df["clean_text"] = df["text"].apply(clean_text)

# TF-IDF-przekształca tekst w macierz liczbową na podstawie miary TF-IDF (Term Frequency-Inverse Document Frequency).
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(df["clean_text"])

y = df["airline_sentiment"].map({"negative": 0, "neutral": 1, "positive": 2}) #Konwersja etykiet sentymentu na liczby

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model regresji logistycznej
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Raport
print("\n📊 Raport klasyfikacji:\n", classification_report(y_test, y_pred))
print("\n🎯 Dokładność modelu:", accuracy_score(y_test, y_pred))

# Analiza wpływowych słów
feature_names = vectorizer.get_feature_names_out()
class_labels = ["negative", "neutral", "positive"]

for i, label in enumerate(class_labels):
    top_words = np.argsort(model.coef_[i])[-10:]
    print(f"\n🔹 Najbardziej wpływowe słowa dla klasy '{label}':")
    print([feature_names[j] for j in top_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tomas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tomas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



📊 Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.81      0.94      0.87      1889
           1       0.65      0.44      0.52       580
           2       0.80      0.59      0.68       459

    accuracy                           0.79      2928
   macro avg       0.75      0.66      0.69      2928
weighted avg       0.78      0.79      0.77      2928


🎯 Dokładność modelu: 0.7855191256830601

🔹 Najbardziej wpływowe słowa dla klasy 'negative':
['hrs', 'delay', 'fix', 'cancelled', 'luggage', 'hold', 'hour', 'delayed', 'worst', 'hours']

🔹 Najbardziej wpływowe słowa dla klasy 'neutral':
['tomorrow', 'question', 'possible', 'winners', 'chance', 'need', 'ceo', 'number', 'photo', 'march']

🔹 Najbardziej wpływowe słowa dla klasy 'positive':
['thx', 'excellent', 'appreciate', 'best', 'love', 'amazing', 'awesome', 'great', 'thanks', 'thank']


In [21]:
tweets = [
    "Once again @airline_name delayed my flight! This is the third time in a row, it’s clear they can't manage their schedule. It's getting really frustrating! 😤",
    "I recently flew with @airline_name and I have to say, the service was amazing! Comfortable seats, great food, and a very friendly crew. I'll definitely fly with them again! ✈️😊",
    "Flew today with @airline_name. The flight went by without any major issues, but nothing really stood out. Just an ordinary trip.",
    "I always enjoy flying with @airline_name, but today I feel like a VIP – all the pilots smiling and the flight attendants are almost too friendly! 😂✈️",
    "Just landed in Paris. No delays, everything went smoothly.",
    "Waiting for my flight to board. The gate area is a bit crowded, but it’s manageable.",
    "Saw an interesting article about air travel today. Looks like there are a lot of new safety regulations.",
    "Got my boarding pass. Still have some time before the flight, so just relaxing.",
    "The airport was pretty busy today, but it’s always like that around this time."
]

def analyze_tweets(tweets):
    sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
    
    for tweet in tweets:
        clean_tweet = clean_text(tweet)  # Oczyszczenie
        vectorized_tweet = vectorizer.transform([clean_tweet])  # Przekształcenie TF-IDF
        prediction = model.predict(vectorized_tweet)[0]  # Przewidywanie sentymentu
        print(f"Tweet: {tweet}")
        print("🔍 Przewidziany sentyment:", sentiment_labels[prediction])
        print("-" * 50)

analyze_tweets(tweets)

Tweet: Once again @airline_name delayed my flight! This is the third time in a row, it’s clear they can't manage their schedule. It's getting really frustrating! 😤
🔍 Przewidziany sentyment: negative
--------------------------------------------------
Tweet: I recently flew with @airline_name and I have to say, the service was amazing! Comfortable seats, great food, and a very friendly crew. I'll definitely fly with them again! ✈️😊
🔍 Przewidziany sentyment: positive
--------------------------------------------------
Tweet: Flew today with @airline_name. The flight went by without any major issues, but nothing really stood out. Just an ordinary trip.
🔍 Przewidziany sentyment: negative
--------------------------------------------------
Tweet: I always enjoy flying with @airline_name, but today I feel like a VIP – all the pilots smiling and the flight attendants are almost too friendly! 😂✈️
🔍 Przewidziany sentyment: positive
--------------------------------------------------
Tweet: Just lan